# Build dataset
## Intergrate latest repos info with ARGUS dataset.

In [1]:
import pandas as pd
import gc
import pprint
from tqdm import tqdm

chunk_size = 100

workflow_df = pd.read_csv('./data/workflows-ir.csv', chunksize=chunk_size)
pd.set_option('display.max_columns', None)

In [ ]:
# clean the ARGUS data by removing the records with empty repository name.

with pd.read_csv('workflows-ir.csv', chunksize=chunk_size, on_bad_lines='skip') as reader:
    for i, chunk in enumerate(reader):
        
        try:
            chunk_cleaned = chunk.dropna(subset=['repo'])
            chunk_cleaned = chunk_cleaned.drop_duplicates()
            chunk_cleaned = chunk_cleaned.reset_index(drop=True)

            if i == 0:
                # Write the first chunk with headers
                chunk_cleaned.to_csv('workflows-ir_cleaned.csv', index=False, mode='w')
            else:
                # Append subsequent chunks without headers
                chunk_cleaned.to_csv('workflows-ir_cleaned.csv', index=False, mode='a', header=False)
            
        except Exception as e:
            print(f"Error in chunk {i}: {e}")
        
        del chunk, chunk_cleaned
        gc.collect()

In [7]:
gitrepos_csv = pd.read_csv('./data/results_19988.csv')
gitrepos_csv.rename(columns={'name': 'repo'}, inplace=True)
gitrepos_csv

,id,repo,isFork,commits,branches,releases,forks,mainLanguage,defaultBranch,license,homepage,watchers,stargazers,contributors,size,createdAt,pushedAt,updatedAt,totalIssues,openIssues,totalPullRequests,openPullRequests,blankLines,codeLines,commentLines,metrics,lastCommit,lastCommitSHA,hasWiki,isArchived,isDisabled,isLocked,languages,labels,topics
0,0,sparklemotion/nokogiri,False,7598,38,68,899,C,main,MIT License,https://nokogiri.org/,161,6130,227.0,37929,2008-07-14T03:34:32,2024-09-03T05:00:58,2024-09-03T05:01:01,1980,63,1206,19,14126.0,126153.0,15023.0,"[{""blankLines"":0,""codeLines"":84,""commentLines""...",2024-09-03T05:00:56,8692abc662c03090229514169a7fac538e0593ab,True,False,False,False,"{""C"":1435135,""Ruby"":1254290,""Java"":660238,""C++...",backport;blocked;dependencies;github_actions;h...,libxml2;libxslt;nokogiri;ruby;ruby-gem;sax;xer...
1,82,tcurdt/jdeb,False,1119,3,0,293,Java,master,Apache License 2.0,https://github.com/tcurdt/jdeb,28,503,64.0,5552,2008-10-03T11:46:26,2024-09-06T06:02:45,2024-09-06T10:14:02,214,28,513,12,1931.0,8172.0,2013.0,"[{""blankLines"":1646,""codeLines"":5772,""commentL...",2024-08-30T01:49:43,8b4b93b31320c33437f9e01c3e842d0adbfb24bf,False,False,False,False,"{""Java"":330757,""Shell"":4598,""PowerShell"":885,""...",bug;dependencies;github_actions;good first iss...,ant-task;cross-platform;deb;debian-packages;go...
2,467,aosp-mirror/platform_frameworks_base,False,1037312,580,0,6202,Java,main,Other,NaN,906,10633,NaN,6655202,2008-10-21T06:20:37,2024-09-08T07:47:55,2024-09-07T01:30:47,0,0,132,8,1072191.0,6492098.0,1816714.0,"[{""blankLines"":75,""codeLines"":672,""commentLine...",2024-09-07T01:11:19,952d16a946806b182c530b22717819453dd71908,False,True,False,False,"{""Java"":244927766,""Kotlin"":33928565,""C++"":1721...",dependencies,NaN
3,470,mongodb/mongo-java-driver,False,8431,42,141,1472,Java,master,Apache License 2.0,https://www.mongodb.com/docs/drivers/java/sync...,218,2611,167.0,131505,2009-01-15T03:16:29,2024-08-30T07:30:15,2024-08-30T11:12:03,0,0,1486,16,58243.0,589799.0,117018.0,"[{""blankLines"":21,""codeLines"":66,""commentLines...",2024-08-30T03:29:03,fc7084d89f77472b0dc0cb720f3ac7e3a40df87d,False,False,False,False,"{""Java"":11476158,""Groovy"":2673462,""Scala"":1319...",kotlin;priority: test,database;java;java-library;jvm;kotlin;kotlin-c...
4,471,unclebob/fitnesse,False,6065,8,10,712,Java,master,Other,fitnesse.org,149,2011,116.0,74305,2008-12-06T07:38:59,2024-08-16T04:18:01,2024-08-16T04:18:05,770,88,743,15,28604.0,148356.0,8955.0,"[{""blankLines"":1229,""codeLines"":5911,""commentL...",2024-08-16T04:18:01,7bed6ad2259a81ba8ef702b419eba042c6f0f1d9,True,False,False,False,"{""Java"":3538008,""JavaScript"":1077028,""Less"":21...",bug;build;dependencies;documentation;faq;featu...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19983,95170870,joshed-io/reveal-hugo,False,349,5,0,141,JavaScript,master,MIT License,https://reveal-hugo.dzello.com/,12,669,25.0,7982,2018-04-26T10:21:56,2024-09-07T10:18:15,2024-09-07T10:18:20,99,34,66,6,3139.0,15118.0,1562.0,"[{""blankLines"":37,""codeLines"":85,""commentLines...",2024-09-07T10:18:15,34938aeb63b2a30f423732253daf0511c4daca0a,True,False,False,False,"{""JavaScript"":63227,""HTML"":45817,""CSS"":24705,""...",bug;dependencies;duplicate;enhancement;good fi...,hugo;presentation;reveal-js;slides;theme
19984,95171570,oban-bg/oban,False,1447,7,47,308,Elixir,main,Apache License 2.0,https://getoban.pro,43,3269,158.0,2706,2019-02-25T03:29:11,2024-09-09T09:25:44,2024-09-09T09:25:46,577,4,571,1,NaN,NaN,NaN,NaN,2024-09-09T09:25:42,6841a30d76e3938c2c400069114f44551d9ca291,False,False,False,False,"{""Elixir"":484568}",area:met;area:oss;area:pro;area:web;closed:dup...,NaN
19985,95171571,mojotech/scrivener_ecto,False,193,3,0,127,Elixir,master,MIT License,NaN,13,559,19.0,153,2016-04-01T03:41:57,2024-09-09T08:03:11,2024-09-09T07:56:22,64,4,46,6,NaN,NaN,NaN,NaN,2024-09-09T

In [5]:
with pd.read_csv('workflows-ir_cleaned.csv', chunksize=chunk_size) as reader:
    for i, chunk in enumerate(reader):
        
        try:
            # Apply the function to extract repo names for each row in the chunk
            chunk_merged = pd.merge(chunk, gitrepos_csv, on='repo')

            # Write the chunk to a new CSV file
            if i == 0:
                # Write the first chunk with headers
                chunk_merged.to_csv('merge_workflows-ir_19988.csv', index=False, mode='w')
            else:
                # Append subsequent chunks without headers
                chunk_merged.to_csv('merge_workflows-ir_19988.csv', index=False, mode='a', header=False)
        
        except Exception as e:
            print(f"Error in chunk {i}: {e}")
        
        del chunk, chunk_merged
        gc.collect()

print("Updated file with repository names saved as updated_large_file.csv")

Error in chunk 24371: You are trying to merge on float64 and object columns. If you wish to proceed you should use pd.concat


NameError: name 'chunk_merged' is not defined

## Download the latest version of the workflows.

In [11]:
import requests
import time
import base64

token = "put github token here."
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github.v3+json"
}

In [5]:
merged_wf = pd.read_csv('./data/merge_workflows-ir_19988.csv')
merged_wf

/var/folders/1s/0z2h15r11fj703zz1p5wg4jc0000gn/T/ipykernel_23287/3785279650.py:1: DtypeWarning: Columns (2,6,66) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_wf = pd.read_csv('./data/merge_workflows-ir_19988.csv')


,_id,analyzed,error,ir.CIvars_set,ir.dependencies,ir.reusable,ir.taskgroups,ir.triggers,ir.type,ir.uid,ir.wf_name,ir.workflow_env,ir.workflow_inputs,ir.workflow_outputs,isInteresting,repo,repo_metadata.allow_forking,repo_metadata.archive_url,repo_metadata.archived,repo_metadata.assignees_url,repo_metadata.blobs_url,repo_metadata.branches_url,repo_metadata.clone_url,repo_metadata.collaborators_url,repo_metadata.comments_url,repo_metadata.commits_url,repo_metadata.compare_url,repo_metadata.contents_url,repo_metadata.contributors_url,repo_metadata.created_at,repo_metadata.default_branch,repo_metadata.deployments_url,repo_metadata.description,repo_metadata.disabled,repo_metadata.downloads_url,repo_metadata.events_url,repo_metadata.fork,repo_metadata.forks,repo_metadata.forks_count,repo_metadata.forks_url,repo_metadata.full_name,repo_metadata.git_commits_url,repo_metadata.git_refs_url,repo_metadata.git_tags_url,repo_metadata.git_url,repo_metadata.has_discussions,repo_metadata.has_downloads,repo_metadata.has_issues,repo_metadata.has_pages,repo_metadata.has_projects,repo_metadata.has_wiki,repo_metadata.homepage,repo_metadata.hooks_url,repo_metadata.html_url,repo_metadata.id,repo_metadata.is_template,repo_metadata.issue_comment_url,repo_metadata.issue_events_url,repo_metadata.issues_url,repo_metadata.keys_url,repo_metadata.labels_url,repo_metadata.language,repo_metadata.languages_url,repo_metadata.license,repo_metadata.merges_url,repo_metadata.milestones_url,repo_metadata.mirror_url,repo_metadata.name,repo_metadata.network_count,repo_metadata.node_id,repo_metadata.notifications_url,repo_metadata.open_issues,repo_metadata.open_issues_count,repo_metadata.organization,repo_metadata.owner,repo_metadata.parent,repo_metadata.permissions,repo_metadata.private,repo_metadata.pulls_url,repo_metadata.pushed_at,repo_metadata.releases_url,repo_metadata.size,repo_metadata.source,repo_metadata.ssh_url,repo_metadata.stargazers_count,repo_metadata.stargazers_url,repo_metadata.statuses_url,repo_metadata.subscribers_count,repo_metadata.subscribers_url,repo_metadata.subscription_url,repo_metadata.svn_url,repo_metadata.tags_url,repo_metadata.teams_url,repo_metadata.temp_clone_token,repo_metadata.template_repository,repo_metadata.topics,repo_metadata.trees_url,repo_metadata.updated_at,repo_metadata.url,repo_metadata.visibility,repo_metadata.watchers,repo_metadata.watchers_count,repo_metadata.web_commit_signoff_required,round,wf_metadata.download_url,wf_metadata.git_url,wf_metadata.html_url,wf_metadata.sha,wf_metadata.size,wf_metadata.type,wf_metadata.url,workflow_name,workflow_path,workflow_yaml,id,isFork,commits,branches,releases,forks,mainLanguage,defaultBranch,license,homepage,watchers,stargazers,contributors,size,createdAt,pushedAt,updatedAt,totalIssues,openIssues,totalPullRequests,openPullRequests,blankLines,codeLines,commentLines,metrics,lastCommit,lastCommitSHA,hasWiki,isArchived,isDisabled,isLocked,languages,labels,topics
0,63c493b77843b612699fd35f,False,NaN,"[{""name"":""CENTCOMM_ROBUST_BENCHMARK_RUNNER_KEY...",NaN,False,NaN,"[{""type"":""workflow_dispatch"",""condition"":""null...",GithubCI,3210f0f0-e22f-44a7-9df9-2e05dd45ef33,Benchmarks,[],[],[],False,space-wizards/space-station-14,True,https://api.github.com/repos/space-wizards/spa...,False,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://github.com/space-wizards/space-station...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/space-wizards/spa...,2017-05-22T01:57:51Z,master,https://api.github.com/repos/space-wizards/spa...,A multiplayer game about paranoia and chaos on...,False,https://api.github.com/repos/space-wizards/spa...,https://api.github.com/repos/spac

In [4]:
def get_workflow_file(workflow_url, headers):
    
    check_access_token(headers)
    
    res = requests.get(workflow_url, headers=headers)
    workflow_file_json = res.json()
    
    return workflow_file_json

In [5]:
def check_access_token(headers):
    
    '''
    check the # of accesses left with a certain GHA token, 
    the program enters sleep mode if the # left is less than 5
    '''
    
    # check if is approaching the token's access limit.
    access_limit = requests.get('https://api.github.com/rate_limit', headers=headers)
    access_limit = access_limit.json()['resources']['core']
    #tqdm.write(access_limit, end='\r', flush = True)
    
    
    if access_limit['remaining'] < 5: 

        current_time = int(time.time())
        sleep_time = access_limit['reset'] - current_time + 2
        print('The program entered a sleep state due to access limit.')
        
        while sleep_time > 0:

            print(f"Remaining sleep time: {sleep_time} seconds", end='\r')
            time.sleep(1)
            sleep_time -= 1
            
            if sleep_time == 0:
                print('Access limit is reset, restart the program.')

    return

In [13]:
# Download the latest workflow files available on GitHub.

workflow_url_list = merged_wf['wf_metadata.url'].tolist()
wf_json_list = []

for wf_url in tqdm(workflow_url_list[4941:]):
    
    try:
        wf_json = get_workflow_file(wf_url, headers)
        wf_json_list.append(wf_json)
    
    except Exception as e:
        pass

wf_df = pd.DataFrame(wf_json_list)
wf_df.to_csv(f'./data/dataframe_current_wfs_all.csv.gz', compression='gzip')

  0%|                                                                                           | 0/19693 [00:00<?, ?it/s]

The program entered a sleep state due to access limit.
Access limit is reset, restart the program.


 25%|███████████████████▋                                                          | 4979/19693 [48:24<2:10:01,  1.89it/s]

The program entered a sleep state due to access limit.
Access limit is reset, restart the program.


 51%|██████████████████████████████████████▍                                     | 9965/19693 [1:48:41<1:23:36,  1.94it/s]

The program entered a sleep state due to access limit.
Access limit is reset, restart the program.


 76%|██████████████████████████████████████████████████████████▍                  | 14940/19693 [2:48:20<41:24,  1.91it/s]

The program entered a sleep state due to access limit.
Access limit is reset, restart the program.


100%|█████████████████████████████████████████████████████████████████████████████| 19693/19693 [3:46:23<00:00,  1.45it/s]


## Complete the workflow dataframe with extra info.

In [65]:
# Load the dataframe.

wf_df = pd.read_csv('./data/dataframe_current_wfs_all.csv.gz', index_col=0)

In [67]:
# Add any missing information to the dataframe using data from the previously merged workflows dataframe.

wf_df['old_sha'] = merged_wf['wf_metadata.sha']
wf_df['lastCommit'] = merged_wf['lastCommit']
wf_df['old_lastCommit'] = merged_wf['repo_metadata.pushed_at']
wf_df['repo'] = merged_wf['repo']

In [68]:
print(f"Number of repositories: {len(list(set(wf_df['repo'].to_list())))}")
print(f"Number of workflows: {len(wf_df)}")

Number of repositories: 5813
Number of workflows: 24634


In [69]:
nan_count = wf_df['sha'].isna().sum()
print(f"There are {nan_count} ({round(nan_count/24634,4)*100}%) YAML files can not be found anymore.")

There are 5671 (23.02%) YAML files can not be found anymore.


In [70]:
# Clean the data by dropping the workflow files w/o SHA number.

wf_df_clean = wf_df.dropna(subset=['sha']).reset_index(drop=True)

In [71]:
identical_count = (wf_df_clean['sha'] == wf_df_clean['old_sha']).sum()
print(f"Among YAML files still exist, {identical_count} out of {24634 - 5671} ({round(identical_count/(24634 - 5671), 4)*100}%) remain unchanged.")

Among YAML files still exist, 2240 out of 18963 (11.81%) remain unchanged.


In [72]:
# Decode contents into original workflow files and add them to the dataframe.

decoded_contents = []

for content in tqdm(wf_df_clean['content'].to_list()):
    
    if type(content) == str:
        decoded_bytes = base64.b64decode(content)
        decoded_yaml = decoded_bytes.decode('utf-8')
        decoded_contents.append(decoded_yaml)
    else:
        decoded_contents.append(float('nan'))
        
wf_df_clean['workflow_yaml'] = decoded_contents
wf_df_clean['old_workflow_yaml'] = merged_wf['workflow_yaml']

100%|██████████████████████████████████████████████████████████████| 18963/18963 [00:00<00:00, 84335.80it/s]


In [73]:
# Convert the local time into UTC time.

from dateutil.parser import isoparse
from pytz import timezone, utc

utc_time = []

for local_dt in wf_df_clean["lastCommit"].to_list():
    local_dt = isoparse(local_dt)
    local_tz = timezone('Europe/Amsterdam')  # Use the Amsterdam timezone
    localized_dt = local_tz.localize(local_dt)
    utc_dt = localized_dt.astimezone(utc)
    final_dt_str = utc_dt.strftime('%Y-%m-%dT%H:%M:%SZ')
    utc_time.append(final_dt_str)
    
wf_df_clean['lastCommit'] = utc_time

In [74]:
# Filter out workflows that are not used for GitHub Actions.

drop_list = [383, 1358, 1685, 2499, 4102, 4194, 4792, 4801, 5979, 6280, 6298, 7781, 7782, 8150, 8583, 10647, 12849, 13545, 13546, 14155, 16466, 16529, 16701, 17146, 18762]
wf_df_clean = wf_df_clean.drop(drop_list).reset_index(drop=True)

In [75]:
print(f"Number of repositories cleaned: {len(list(set(wf_df_clean['repo'].to_list())))}")
print(f"Number of workflows cleaned: {len(wf_df_clean)}")

Number of repositories cleaned: 5246
Number of workflows cleaned: 18938


In [76]:
# Save the completed dataframe to CSV file.

wf_df_clean.to_csv(f'./data/dataframe_current_wfs_raw_cleaned.csv.gz', compression='gzip')